In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import GBTClassifier
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
28,application_1579214001909_0029,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Selected Data

In [3]:
train_df = ss.read.parquet("s3a://usf.msds.stillbikeshare/noutlier_train")
valid_df = ss.read.parquet("s3a://usf.msds.stillbikeshare/noutlier_test")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
va = VectorAssembler(outputCol="features", inputCols=train_df.columns[0:-1])
train_va = va.transform(train_df).select("features", "label")
train_va.cache()

test_va = va.transform(valid_df).select("features", "label")
test_va.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[features: vector, label: int]

In [5]:
f_train, f_valid = train_va.randomSplit([0.8, 0.2])
f_train.cache()
f_valid.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[features: vector, label: int]

In [6]:
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.regression import DecisionTreeRegressor
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(train_va)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
dt = DecisionTreeRegressor(featuresCol="indexedFeatures", maxBins = 100, maxDepth=5)
pipeline = Pipeline(stages=[featureIndexer, dt])
model = pipeline.fit(f_train)
predictions = model.transform(f_valid)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 1486.55

In [8]:
dt = DecisionTreeRegressor(featuresCol="indexedFeatures", maxBins = 100, maxDepth=10)
pipeline = Pipeline(stages=[featureIndexer, dt])
model = pipeline.fit(f_train)
predictions = model.transform(f_valid)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 1505.73

In [9]:
dt = DecisionTreeRegressor(featuresCol="indexedFeatures", maxBins = 100, maxDepth=3)
pipeline = Pipeline(stages=[featureIndexer, dt])
model = pipeline.fit(f_train)
predictions = model.transform(f_valid)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 1500.67

In [10]:
dt = DecisionTreeRegressor(featuresCol="indexedFeatures", maxBins = 100, maxDepth=3)
pipeline = Pipeline(stages=[featureIndexer, dt])
model = pipeline.fit(train_va)
predictions = model.transform(test_va)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 1346.74

### Full Data

In [20]:
train_df_full = ss.read.parquet("s3://usf.msds.stillbikeshare/full_train")
valid_df_full = ss.read.parquet("s3://usf.msds.stillbikeshare/full_test")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
va = VectorAssembler(outputCol="features", inputCols=train_df_full.columns[1:])
train_va_full = va.transform(train_df_full).select("features", "label")
train_va_full.cache()

test_va_full = va.transform(valid_df_full).select("features", "label")
test_va_full.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[features: vector, label: int]

In [29]:
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(train_va_full)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
dt_full = DecisionTreeRegressor(featuresCol="indexedFeatures", maxBins = 100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
pipeline = Pipeline(stages=[featureIndexer, dt_full])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
model = pipeline.fit(train_va_full)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
predictions = model.transform(test_va_full)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) on test data = 4489.51